In [1]:
library(lme4)
library(sjPlot)
library(Hmisc)

library(gridExtra)
library(ggplot2)

# custom style for tab models
css = list(
    css.thead = 'border-top: double; text-align:center; font-style:normal; font-weight:bold; padding:0.1cm;',
    css.tdata = 'padding:0.05cm; padding-left:0.2cm; padding-right:0.2cm; text-align:left; vertical-align:top;',
    css.depvarhead = 'text-align:center; border-bottom:1px solid; font-style:italic; font-weight:bold',
    css.summary = 'padding-top:0.05cm; padding-bottom:0.05cm;'
  )

Loading required package: Matrix

#refugeeswelcome

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:base’:

    format.pval, units




# Read Data

In [2]:
wtdata <- read.csv("data/dataset_A_linked.csv")

# filter the relevant analysis period
wtdata = wtdata[wtdata$is_analysis_period,]

In [3]:
colnames(wtdata)

[1] "search_page"             "election_period_cat"    
 [3] "total_results_per_page"  "rank"                   
 [5] "is_selected_result"      "coredomain"             
 [7] "news_representativeness" "is_analysis_period"     
 [9] "visits_TI_B1"            "allnews_visits_TI_B1"   
[11] "engagement_TI_B1"        "visits_TI_C1"           
[13] "allnews_visits_TI_C1"    "engagement_TI_C1"       
[15] "visits_TI_D1"            "allnews_visits_TI_D1"   
[17] "engagement_TI_D1"        "visits_TD_B"            
[19] "allnews_visits_TD_B"     "engagement_TD_B"        
[21] "visits_TD_C"             "allnews_visits_TD_C"    
[23] "engagement_TD_C"         "visits_TD_D"            
[25] "allnews_visits_TD_D"     "engagement_TD_D"        
[27] "participant"             "age"                    
[29] "education"               "political_alignment"    
[31] "is_west_germany"         "is_female"              
[33] "is_diverse"              "age_val"                
[35] "education_val"           "political_alignment_val"
[37] "is_west_germany_val"     "is_female_val"          
[39] "is_diverse_val"

# Scale and center variables

In [4]:
# scale and center the election periods
wtdata$is_before_elections <- scale(wtdata$election_period_cat == 'A', scale=T)
wtdata$is_during_elections <- scale(wtdata$election_period_cat == 'B', scale=T)
wtdata$is_after_elections <- scale(wtdata$election_period_cat == 'C', scale=T)

# scaling the rest of the variables
wtdata$rank <- scale(wtdata$rank, scale=T)
# invert this value so that the odds ratio, so it the scaled scores
# match the intuition, i.e. the 1st result should have a higher (positive)
# value. The odds ratio are then easier to interpret.
wtdata$inv_rank <- -1 * wtdata$rank

# scale and center the representativeness
wtdata$news_representativeness <- scale(wtdata$news_representativeness, scale=T)

# scale and center engagement variables
wtdata$engagement_TI_B1 <- scale(wtdata$engagement_TI_B1, scale=T)
wtdata$engagement_TI_C1 <- scale(wtdata$engagement_TI_C1, scale=T)
wtdata$engagement_TI_D1 <- scale(wtdata$engagement_TI_D1, scale=T)

# scale and center engagement variables
wtdata$visits_TI_B1 <- scale(wtdata$visits_TI_B1, scale=T)
wtdata$visits_TI_C1 <- scale(wtdata$visits_TI_C1, scale=T)
wtdata$visits_TI_D1 <- scale(wtdata$visits_TI_D1, scale=T)

# scale and center engagement variables
wtdata$allnews_visits_TI_B1 <- scale(wtdata$allnews_visits_TI_B1, scale=T)
wtdata$allnews_visits_TI_C1 <- scale(wtdata$allnews_visits_TI_C1, scale=T)
wtdata$allnews_visits_TI_D1 <- scale(wtdata$allnews_visits_TI_D1, scale=T)

# convert to factors the within subjects
wtdata$search_page <- as.factor(wtdata$search_page)
wtdata$participant <- as.factor(wtdata$participant)
wtdata$coredomain <- as.factor(wtdata$coredomain)

# Dataset B1: previous engagement

In [5]:
# time independent
ti_model_B1 <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TI_B1
                            * inv_rank
                            * news_representativeness 
                     
                            # control by total news visits
                            + allnews_visits_TI_B1

                            # control for demographics
                            + is_female
                            + is_diverse
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)

tab_model(ti_model_B1, wrap.labels = F, CSS = css, file = "statistical_tests/absolute_visits/ti_model_B1.html")

boundary (singular) fit: see help('isSingular')



# Dataset C1: Google associated previous engagement

In [6]:
# time independent
ti_model_C1 <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TI_C1
                            * inv_rank
                            * news_representativeness 
                     
                            # control by total news visits
                            + allnews_visits_TI_C1

                            # control for demographics
                            + is_female
                            + is_diverse
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)

tab_model(ti_model_C1, wrap.labels = F, CSS = css, file = "statistical_tests/absolute_visits/ti_model_C1.html")


boundary (singular) fit: see help('isSingular')



# Dataset D1: non Google associated previous engagement

In [7]:
# time independent
ti_model_D1 <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TI_D1
                            * inv_rank
                            * news_representativeness 
                     
                            # control by total news visits
                            + allnews_visits_TI_D1

                            # control for demographics
                            + is_female
                            + is_diverse
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)


tab_model(ti_model_D1, wrap.labels = F, CSS = css, file = "statistical_tests/absolute_visits/ti_model_D1.html")

boundary (singular) fit: see help('isSingular')

